# Plot Na MOT Power Logs
Plot and analyze Na MOT Power history extracted from pickoff photodetectors (PDs).
Need to have:
- `rootfolder`: path to photodetector calibration parameter table that converts PD voltage (in V) to laser power (in mW). One can use the helper script `'...\instrument\mot_pd_calibration.py'` to create such a parameter table.
- `rootfolder`: path to a folder you want to plot. The notebook will automatically look through all levels of subfolders to find folders that contain image files from the experiment. The notebook then takes the `run_id` of those iamge files and retreive the corresponding dataframe (as `df`) from the breadboard. 
    - Additionally one can specify the subfolders under the rootfolder to only plot data associated with those subfolders, To do this, set `selected_folders` as a list of subfolder names. Otherwise set `selected_folders` to `None`. 

### Filepath and helper functions

In [ ]:
from utility_functions import get_newest_df, load_bec1serverpath
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import re
import itertools
import seaborn as sns
rootfolder = r'Z:\Fermicam 1 data (zcam)\2022\202209'
selected_folders = ['220915'] # specify subfolders under the rootfolder if you want to only plot results from those folders
# selected_folders = None  # alternatively leave as None to select all sub folders under the rootfolder
calibration_filepath = r'instruments\MOT Monitor Calibration Parameter Table_2022-09-15.csv'

In [ ]:
def get_run_dirs(root, selected_dirs = False):
    """
    Get the paths of watchfolders that you want to use to check the MOT powers

    Args: 
        -root: path str of a parent directory. 
        -selected_dirs: LIST of str where each str is the directory name of a selected sub directroy 
        in the parent directory or leave as False to iterate through all subdirectories

    Return: a list of paths of run folders

    """
    from glob import glob
    run_dirs = []
    if selected_dirs == False:
        selected_dirs = [dir for dir in os.listdir(root)]

    #if provided root path is a specific experimental run: 
    if any('.spe' in dir for dir in selected_dirs):
        run_dirs = [root]
    #if provided root path is a specific day:
    elif any('run' in dir for dir in selected_dirs): 
        for dir in selected_dirs:
            if 'run' in dir:
                path = os.path.abspath(os.path.join(root, dir))
                run_dirs += [path]
    #if provided root path is a specific month or above:
    else:
        for dir in selected_dirs:
            path = os.path.abspath(os.path.join(root, dir))
            run_dirs += glob(path + r'/**/run*/', recursive=True)

    return run_dirs    
    
def calibrate_MOT_monitor(df, calibration_filepath):
    df_fits = pd.read_csv(calibration_filepath, index_col=0)
    for port in df_fits.columns:
        func = np.poly1d(df_fits[port])
        voltages = df[f'NaMOT_{port}_mean_in_mv']/1000
        powers  = func(voltages)
        df[f'NaMOT_{port}_mean_in_mw'] = powers

def show_MOT_stat(df):
    mean = df[NaMOT_ports+['Na_TFx']].mean()
    std = df[NaMOT_ports+['Na_TFx']].std()
    percent_drift = 100*std/mean
    port_label = [port.split('_')[1] for port in NaMOT_ports] + ['Na_TFx']
    df_stat = pd.concat([mean,percent_drift],axis=1)
    df_stat.index = port_label
    df_stat.columns = ['Mean Laser Power [mW]', '% Standard Deviation']
    return df_stat

def show_MOT_hist(df):
    data = df[NaMOT_ports]
    labels = [label.split('_')[1] for label in NaMOT_ports]
    data.columns = labels
    
    fig, axes = plt.subplots(1, 7, figsize=(20,4))
    data.hist(bins = 10, ax = axes, color='orange', rwidth=0.9)
    for ax, label in zip(axes, labels):
        ax.set_xlabel('Laser Power [mW]')
        ax.set_ylabel('Counts')
    ax.grid(color='lightgray', alpha=0.8)
    plt.tight_layout()


def filter_by_power(df, nsigma = 5):
    """Filter bad shots with very low total MOT powers by nsigma standard deviation"""
    mean, std = df['NaMOT_Total_mean_in_mw'].mean(), df['NaMOT_Total_mean_in_mw'].std()
    df_filtered = df[np.abs(df['NaMOT_Total_mean_in_mw'] - mean) <= nsigma*std]
    ntotal = len(df.index)
    nfiltered = ntotal - len(df_filtered.index)
    print(f'Removed {nfiltered} outliers out of {ntotal} with total powers {nsigma} std away from mean')
    return df_filtered

def filter_by_BECsize(df, nsigma = 5):
    """Filter bad shots with very low Na_TFx by nsigma standard deviation"""
    mean, std = df['Na_TFx'].mean(), df['Na_TFx'].std()
    df_filtered = df[np.abs(df['Na_TFx'] - mean) <= nsigma*std]
    ntotal = len(df.index)
    nfiltered = ntotal - len(df_filtered.index)
    print(f'Removed {nfiltered} outliers out of {ntotal} with Na_TFx {nsigma} std away from mean')
    return df_filtered

### Retrieve and clean up data from breadboard

In [ ]:
watchfolders = get_run_dirs(rootfolder,selected_folders)
print('Found ' + str(len(watchfolders)) + ' run folders to monitor Na MOT powers')

In [ ]:
df_raw = pd.concat([get_newest_df(folder) for folder in watchfolders])
print('Imported dataframe shape: '+ str(df_raw.shape))

In [ ]:
#Clean up: indexing, datetime formatting, ordering
df_raw.index = df_raw['run_id']
times = [datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ") for timestamp in df_raw['runtime (format: %Y-%m-%dT%H:%M:%SZ)']]
df_raw['datetimes'] = times
df_raw = df_raw.sort_values(by='datetimes')

In [ ]:
#Convert MOT montor photodetector voltage to laser power
calibrate_MOT_monitor(df_raw, calibration_filepath)
NaMOT_ports = [k for k in df_raw.columns if re.search('NaMOT(.*)mean_in_mw',k)]
df_raw['NaMOT_Total_mean_in_mw'] = df_raw[NaMOT_ports].sum(axis=1, min_count=1)
NaMOT_ports += ['NaMOT_Total_mean_in_mw']

In [ ]:
#Filter outliers by total mot powers to get rid of accidental bad shots
df = df_raw.copy()
df = filter_by_power(df=df_raw, nsigma=3)

In [ ]:
#Filter outliers by Na_TFx to get rid of bad shots
df = filter_by_BECsize(df=df, nsigma=3)

In [ ]:
#Normalize time series to see drifts in percentages. TODO: Handle NaN values of Na_TFx
startlogging_idx = df['NaMOT_Top_mean_in_mw'].index.get_loc(df['NaMOT_Top_mean_in_mw'].first_valid_index())
print('Normalizing all relevant time series with the following starting values: ')
print(df[NaMOT_ports+['Na_TFx']].iloc[startlogging_idx].to_string())

### Summary statistics and plots

#### Statistics

In [ ]:
NaMOT_stats = show_MOT_stat(df)
NaMOT_stats

In [ ]:
show_MOT_hist(df)

#### All traces with powers in mW

In [ ]:
plt.rcParams.update({'font.size': 18})
fig, [ax1,ax2, ax3] = plt.subplots(3,1,figsize=(30,16),sharex=True, gridspec_kw={'height_ratios': [4, 0.5, 0.5]})
for port in NaMOT_ports[:-1]:
    label = port.split('_')[1]
    ax1.scatter(df['datetimes'],df[port],label=label)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Laser Power [mW]')
    ax1.xaxis.set_tick_params(labelbottom=True)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%y/%m/%d %H:%M'))
    ax1.grid(color='lightgray', alpha=0.8)
    ax1.set_title('Na MOT Power Monitoring: Six Ports')


ax2.scatter(df['datetimes'],df['NaMOT_Total_mean_in_mw'],color='darkred')
ax2.xaxis.set_tick_params(labelbottom=True)
ax2.set_xlabel('Time')
ax2.set_ylabel('Laser Power [mW]')
ax2.grid(color='lightgray', alpha=0.8)
ax2.set_title('Na MOT Power Monitoring: Total')

ax3.scatter(df['datetimes'],df['Na_TFx'],color='midnightblue')
ax3.xaxis.set_tick_params(labelbottom=True)
ax3.set_xlabel('Time')
ax3.set_ylabel('Na_TFx')
ax3.grid(color='lightgray', alpha=0.8)
ax3.set_title('Na MOT Power Monitoring: BEC Size (Na_TFx)')

fig.legend(loc='right', ncol = 1, title='MOT Ports')
plt.tight_layout()
# mpld3.display()

#### Individual traces with powers in %

In [ ]:
fig, axes = plt.subplots(8,1,sharex=True,figsize=(27,18))
colors = ['crimson'] * 6 + ['darkred','midnightblue']
for port, ax, c in zip(NaMOT_ports, axes[:-1], colors[:-1]):
    label = port.split('_')[1]
    start_value = df[port].iloc[startlogging_idx]
    (mean, std) = NaMOT_stats.loc[label,:]
    ax.scatter(df['datetimes'],df[port]/start_value,label=label,color=c)
    ax.xaxis.set_tick_params(labelbottom=True)
    ax.grid(color='lightgray', alpha=0.8)
    ax.set_xlabel('Time')
    ax.set_ylabel('Power')
    ax.set_title(f'Na MOT - {label} (Normalized)    Mean = {mean:.1f}mW   std = {std:.2f}%')

(mean_TFx, std_TFx) = NaMOT_stats.loc['Na_TFx',:]
axes[-1].scatter(df['datetimes'], df['Na_TFx']/df['Na_TFx'].iloc[startlogging_idx], label='Na_TFx',color=colors[-1])
axes[-1].grid(color='lightgray', alpha=0.8)
axes[-1].set_xlabel('Time')
axes[-1].set_ylabel('Na_TFx')
axes[-1].set_title(f'Na_TFx (Normalized)    Mean = {mean_TFx:.1f}mW   std = {std_TFx:.2f}%')
axes[-1].xaxis.set_major_formatter(mdates.DateFormatter('%y/%m/%d %H:%M'))

# axes[0].set_xlim([datetime(2022,9,13,13,30), datetime(2022,9,13,19,10)])
plt.tight_layout()


### More plots: correlation analysis

In [ ]:
fig, ax = plt.subplots(figsize=(40,10))
ax.scatter(df['datetimes'], df['NaMOT_NaSlower_mean_in_mw']/df['NaMOT_NaSlower_mean_in_mw'].iloc[startlogging_idx], 
            label='Na MOT_NaSlower Power (Normalized)',
            color='red',
            s=50
            )
ax.scatter(df['datetimes'], df['Na_TFx']/df['Na_TFx'].iloc[startlogging_idx], label='Na_TFx (Normalized)',color='blue')
ax.plot(df['datetimes'], df['Na_TFx']/df['Na_TFx'].iloc[startlogging_idx], label='Na_TFx (Normalized)',color='gray')

# ax.set_xlim([datetime(2022,9,13,13,30), datetime(2022,9,13,19,10)])
ax.xaxis.set_major_formatter(mdates.DateFormatter('%y/%m/%d %H:%M'))
ax.set_xlabel('Time')
ax.set_ylabel('Normalized Things')
ax.set_title('Correlation between MOT power and BEC Size')
fig.legend(loc='upper center', ncol = 3)
ax.grid(color='lightgray', alpha=0.8)

# mpld3.display(fig)

In [ ]:
plt.rcParams.update({'font.size': 12})
fig, axes = plt.subplots(3,3,sharey=True,figsize=(20,10))
for port, ax in zip(NaMOT_ports, axes.flat[:7]):
    label = port.split('_')[1]
    fx = np.poly1d(np.polyfit(df[port],df['Na_TFx'],1))
    p_min, p_max = df[port].min(), df[port].max()
    powers = np.linspace(p_min,p_max,100)

    ax.scatter(df[port], df['Na_TFx'],color='blue')
    ax.plot(powers,fx(powers),label=f'Linear Fit: {fx}', color='red',lw=2)
    ax.grid(color='lightgray', alpha=0.8)
    ax.set_xlabel('Laser Power [mW]')
    ax.set_ylabel('Na_TFx')
    ax.set_title(f'{label}')
    ax.legend()
fig.delaxes(axes[2][2])
fig.delaxes(axes[2][1])
fig.suptitle('Correlation between BEC size and each MOT port power')
plt.tight_layout()

In [ ]:
corrmat = df[['Na_TFx'] + NaMOT_ports].corr().round(5)
corrmat.columns = (['Na_TFx']+[port.split('_')[1] for port in NaMOT_ports])
corrmat.index = (['Na_TFx']+[port.split('_')[1] for port in NaMOT_ports])
plt.rcParams.update({'font.size': 10})
fig, ax = plt.subplots(figsize=(8,8))
ax.set_title('Correlation Matrix')
sns.heatmap(corrmat, annot=True, fmt='.3f',linewidths=0.5, ax=ax)

In [ ]:
# fig, ax = plt.subplots(figsize=(30,10))
# for port in NaMOT_ports:
#     label = port.split('_')[1]
#     start_value = df[port].iloc[startlogging_idx]
#     ax.scatter(df['datetimes'],df[port]/start_value,label=label)
# ax.scatter(df['datetimes'], df['Na_TFx']/df['Na_TFx'].iloc[startlogging_idx], label='Na_TFx',marker='x')
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%y/%m/%d %H:%M'))

# # ax.set_xlim([datetime(2022,9,13,13,30), datetime(2022,9,13,19,10)])

# ax.set_xlabel('Time')
# ax.set_ylabel('Normalized Things')
# ax.set_title('Na MOT Power Monitoring - Normalized by Starting Value')
# fig.legend(loc='upper center', ncol = 8)
# ax.grid(color='lightgray', alpha=0.8)

# # mpld3.display(fig)